# # 🚀 Sistema de copy trading

In [1]:
import asyncio
import os
from typing import Dict
from dotenv import load_dotenv # type: ignore

from logging_system import setup_logging

from copy_trading import (
    CopyTrading,
    CopyTradingConfig,
    TraderConfig,
    TraderInfo,
    NicknameGenerator,
    AmountMode,
    TransactionType
)

load_dotenv()

setup_logging(
    console_output=True,
    file_output=True,
    min_level_to_process='DEBUG',
    module_levels={}
)

🎯 Módulo Copy Trading Mini cargado
🚀 Sistema simplificado de copy trading
   ✅ Integración directa con PumpFun
   ✅ Replicación automática de trades
   ✅ Gestión de posiciones FIFO
   ✅ Validaciones configurables
   ✅ Logging estructurado
   ✅ Soporte Lightning y Local Trade
   ✅ Sistema de notificaciones mejorado
   ✅ Herramientas de diagnóstico
   ✅ Diagnóstico de I/O de archivos
   ✅ TokenTraderManager optimizado


In [2]:
# Configuración de traders
balance_to_invest = "0.022552" # SOL

traders_to_copy = [
    TraderInfo(
        wallet_address="GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz",
        nickname_generator=NicknameGenerator.CUSTOM,
        nickname="Trader A"
    ),
    TraderInfo(
        wallet_address="EJycPTHve5wgRkLyf6BzwjrSi4HUnqUkThYfwpegrGei",
        nickname_generator=NicknameGenerator.CUSTOM,
        nickname="Trader B"
    )
]

traders_config: Dict[str, TraderConfig] = {
    "GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz": TraderConfig(
        trader_info=traders_to_copy[0],
        enabled=True,

        amount_mode=AmountMode.FIXED,
        amount_value="0.005",

        max_amount_to_invest="0.011",
        max_open_tokens=2,
        max_open_positions_per_token=1,
        use_balanced_allocation=True,
        min_position_size=None,
        max_position_size=None,
        adjust_position_size=True,
        max_daily_volume_sol_open=None,
        min_trade_interval_seconds=1,
    ),
    "EJycPTHve5wgRkLyf6BzwjrSi4HUnqUkThYfwpegrGei": TraderConfig(
        trader_info=traders_to_copy[1],
        enabled=True,

        amount_mode=AmountMode.FIXED,
        amount_value="0.005",

        max_amount_to_invest="0.011",
        max_open_tokens=2,
        max_open_positions_per_token=1,
        use_balanced_allocation=True,
        min_position_size=None,
        max_position_size=None,
        adjust_position_size=True,
        max_daily_volume_sol_open=None,
        min_trade_interval_seconds=1,
    )
}

copy_trading.config - DEBUG - TraderInfo inicializado para wallet: GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz, nickname: Trader-A
copy_trading.config - DEBUG - TraderInfo inicializado para wallet: EJycPTHve5wgRkLyf6BzwjrSi4HUnqUkThYfwpegrGei, nickname: Trader-B
copy_trading.config - DEBUG - TraderConfig inicializado para trader: GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz, enabled: True
copy_trading.config - DEBUG - TraderConfig inicializado para trader: EJycPTHve5wgRkLyf6BzwjrSi4HUnqUkThYfwpegrGei, enabled: True


In [3]:
async def main():
    """Función principal para ejecutar el sistema con notificaciones."""
    # 1. Configurar las credenciales de Telegram
    # Puedes obtener estas credenciales de:
    # - Bot Token: Crear un bot con @BotFather en Telegram
    # - Chat ID: Enviar mensaje a tu bot y usar la API de Telegram para obtener el chat_id

    telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN")  # Token del bot
    telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")     # ID del chat

    if not telegram_bot_token or not telegram_chat_id:
        print("⚠️ ADVERTENCIA: Credenciales de Telegram no configuradas")
        print("   Configura las variables de entorno:")
        print("   - TELEGRAM_BOT_TOKEN: Token de tu bot de Telegram")
        print("   - TELEGRAM_CHAT_ID: ID del chat donde recibir notificaciones")
        print("   El sistema funcionará sin notificaciones de Telegram")

    # 2. Configurar traders a seguir
    """ traders_to_copy = [
        #"3q1WCMp25FQcs1JV86H5Y7XPfxjudiMudtsCBGfgCwj7", # Ejemplo de trader 1
        #"8HEFiQSKqmEJ65UVEmHEyjvieZhrHRdLM7N4KuKpLS3Y", # Ejemplo de trader 2
        #"A1Uh8udZg2CMHS5kKeQouYRGfiAxJ7MYMuRbAz913hJX", # Ejemplo de trader 3
        "GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz", # Ejemplo de trader 4
    ] """

    # 3. Crear configuración con notificaciones habilitadas
    config = CopyTradingConfig(
        traders=traders_to_copy,
        trader_configs=traders_config,

        wallet_file="wallets/wallet_pumpportal.json",

        general_available_balance_to_invest=balance_to_invest,

        # Configuración de copy trading
        amount_mode=AmountMode.EXACT,
        amount_value="10.0",  # 10% de cada trade

        max_traders_per_token=1,

        # Configuración de notificaciones
        notifications_enabled=True,
        telegram_bot_token=telegram_bot_token,
        telegram_chat_id=telegram_chat_id,
        telegram_messages_per_minute=30,  # Límite de mensajes por minuto

        # Configuración de trading
        transaction_type=TransactionType.LIGHTNING_TRADE,
        pool_type="auto",
        slippage_tolerance=15,  # 15%
        priority_fee_sol=0.00005,

        # Modo de prueba
        dry_run=False  # Cambiar a False para trading real
    )

    # 4. Crear e iniciar el sistema
    system = CopyTrading(config=config)

    try:
        async with system:
            print(f"\n✅ Sistema iniciado con notificaciones")
            print(f"📱 Telegram: {'Habilitado' if telegram_bot_token and telegram_chat_id else 'Deshabilitado'}")
            print(f"🔄 Modo: {'DRY RUN' if config.dry_run else 'LIVE'}")
            print(f"👥 Siguiendo {len(config.traders)} traders")
            print("\nPresiona Ctrl+C para detener.")

            # Enviar notificación de prueba
            if system.notification_manager:
                status_msg = (
                    f"Sistema iniciado correctamente\n"
                    f"- Telegram: {'✅ Habilitado' if telegram_bot_token and telegram_chat_id else '❌ Deshabilitado'}\n"
                    f"- Modo: {'🔄 DRY RUN' if config.dry_run else '🚀 LIVE'}\n"
                    f"- Traders: 👥 {len(config.traders)}"
                )
                await system.notification_manager.notify_system(status_msg, "success")

            # Bucle principal - mostrar métricas cada 5 minutos
            while system.is_running:
                await asyncio.sleep(300)  # 5 minutos

                try:
                    metrics = await system.get_metrics()

                    print("\n--- Métricas del Sistema ---")
                    print(f"  - Tiempo activo: {metrics['system_metrics']['uptime_seconds']:.0f}s")
                    print(f"  - Trades recibidos: {metrics['callback_stats']['trades_received']}")
                    print(f"  - Trades validados: {metrics['callback_stats']['trades_validated']}")
                    print(f"  - Trades en cola: {metrics['queue_stats']['pending_count']}")
                    print(f"  - Trades ejecutados: {metrics['system_metrics']['trades_executed']}")
                    print(f"  - Volumen total: {metrics['system_metrics']['total_volume_sol']:.6f} SOL")
                    print(f"  - Balance actual: {metrics['wallet_balance']:.6f} SOL")
                    print("--------------------------\n")

                    # Enviar resumen por Telegram cada hora
                    if system.notification_manager and metrics['system_metrics']['uptime_seconds'] % 3600 < 300:
                        metrics_msg = (
                            f"Resumen horario del sistema\n"
                            f"- Tiempo activo: ⏱️ {metrics['system_metrics']['uptime_seconds']/3600:.1f}h\n"
                            f"- Trades recibidos: 📥 {metrics['callback_stats']['trades_received']}\n"
                            f"- Trades ejecutados: ✅ {metrics['system_metrics']['trades_executed']}\n"
                            f"- Volumen: 💰 {metrics['system_metrics']['total_volume_sol']:.6f} SOL\n"
                            f"- Balance: 💎 {metrics['wallet_balance']:.6f} SOL"
                        )
                        await system.notification_manager.notify_system(metrics_msg, "info")

                except Exception as e:
                    print(f"Error obteniendo métricas: {e}")

    except KeyboardInterrupt:
        print("\nInterrupción del usuario detectada.")
    except Exception as e:
        print(f"Error en el sistema: {e}")

        # Notificar error crítico
        if system.notification_manager:
            error_msg = (
                f"Error crítico en el sistema:\n"
                f"- Tipo: ⚠️ {type(e).__name__}\n"
                f"- Mensaje: 🔥 {str(e)}"
            )
            await system.notification_manager.notify_system(error_msg, "error")

    print("🏁 Sistema detenido.")

In [4]:
try:
    await main()
except asyncio.CancelledError:
    print("Programa cancelado por el usuario")

copy_trading.config - INFO - Inicializando configuración de Copy Trading
copy_trading.config - DEBUG - Configuración inicializada con 2 traders
CopyTrading - INFO - Inicializando sistema Copy Trading
TradingDataFetcher - DEBUG - TradingDataFetcher inicializado
CopyTrading - DEBUG - TradingDataFetcher inicializado
CacheManager - DEBUG - CacheManager inicializado - Token TTL: 300s, Trader TTL: 600s
TokenTraderManager - DEBUG - TokenTraderManager inicializado
CopyTrading - DEBUG - TokenTraderManager inicializado
TelegramStrategy - DEBUG - TelegramStrategy inicializada con chat_id: -4943557468
TelegramStrategy - DEBUG - Worker de Telegram iniciado
CopyTrading - DEBUG - Estrategia de Telegram configurada
ConsoleStrategy - DEBUG - ConsoleStrategy inicializada
CopyTrading - DEBUG - Estrategia de consola configurada
NotificationManager - DEBUG - Estrategia agregada: ConsoleStrategy
NotificationManager - DEBUG - NotificationManager inicializado con 1 estrategias
CopyTrading - DEBUG - Sistema de